# DeepPhys: rPPG Estimation

This notebook demonstrates the use of the DeepPhys model for remote photoplethysmography (rPPG) estimation. The model is based on the paper [DeepPhys: Video-Based Physiological Measurement Using Convolutional Attention Networks](https://arxiv.org/abs/1805.07888) by Weixuan Chen and Daniel McDuff. The model is implemented in PyTorch.

The rPPG signal is later used to estimate the respiratory rate of the subject.

## Load the DeepPhys model

In [ ]:
import torch
import respiration.utils as utils

from respiration.extractor.deep_phys import DeepPhys

device = utils.get_torch_device()

dim = 72

model = DeepPhys(img_size=dim)
model = torch.nn.DataParallel(model).to(device)

model_path = utils.file_path('data', 'rPPG-Toolbox', 'BP4D_PseudoLabel_DeepPhys.pth')
model.load_state_dict(torch.load(model_path, map_location=device))

model = model.module.to(device)

## Load the test video

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

subject = 'Proband21'
setting = '101_natural_lighting'

frames, meta = dataset.get_video_rgb(subject, setting, show_progress=True)

pleth = dataset.get_vital_sign(subject, setting, utils.VitalSigns.pleth)
breath = dataset.get_vital_sign(subject, setting, utils.VitalSigns.thorax_abdomen)

In [ ]:
import respiration.extractor.mtts_can.preprocess as preprocess

raw, diff = preprocess.preprocess_video_frames(frames, dim)

# Permute from (T, H, W, C) to (T, C, H, W)
diff = torch.tensor(diff).permute(0, 3, 1, 2)
raw = torch.tensor(raw).permute(0, 3, 1, 2)

print(diff.shape, raw.shape)

# Stack the two channels
frames_chunk = torch.cat((diff, raw), dim=1).to(device)
frames_chunk.shape

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(20, 6))

# Permute from (C, H, W) to (H, W, C)
diff_frame = diff[0].permute(1, 2, 0)
raw_frame = raw[0].permute(1, 2, 0)

# Normalize the frames
diff_frame = (diff_frame - diff_frame.min()) / (diff_frame.max() - diff_frame.min())
raw_frame = (raw_frame - raw_frame.min()) / (raw_frame.max() - raw_frame.min())

axs[0].imshow(diff_frame)
axs[0].set_title('Diff channel')

axs[1].imshow(raw_frame)
axs[1].set_title('Raw channel')

plt.show()

In [ ]:
with torch.no_grad():
    output = model(frames_chunk)

prediction = output.cpu().detach().numpy().squeeze()
prediction.shape

In [ ]:
import numpy as np
import respiration.analysis as analysis

comparator = analysis.SignalComparator(
    prediction,
    pleth[1:len(prediction) + 1],
    sample_rate=meta.fps,
    lowpass=0.7,
    highpass=2.5,
    detrend_tarvainen=False,
    filter_signal=True,
    normalize_signal=True,
)

In [ ]:
comparator.errors()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 6))

# Plot the predicted rPPG signal
plt.plot(comparator.prediction, label='Predicted rPPG signal')
plt.plot(comparator.ground_truth, label='Ground truth pleth signal')

plt.title('rPPG estimation using DeepPhys')
plt.xlabel('Frame')
plt.ylabel('Signal value')

plt.legend()
plt.show()

## Use sliding window to estimate the error and correlation

In [ ]:
import respiration.preprocessing as preprocessing

prediction_filtered = preprocessing.butterworth_filter(prediction, meta.fps, order=6, lowpass=0.7, highpass=2.5)

In [ ]:
import respiration.analysis as analysis

# Divide the signal into 30-second windows with a 1-second stride
window_size = 30 * meta.fps
stride = meta.fps

prediction_results = []

for inx in range(0, len(prediction_filtered) - window_size, stride):
    prediction_window = prediction_filtered[inx:inx + window_size]
    gt_window = pleth[inx:inx + window_size]

    freq_pred = analysis.frequency_from_psd(prediction_window, meta.fps)
    freq_gt = analysis.frequency_from_psd(gt_window, meta.fps)

    prediction_results.append({
        'freq_pred': freq_pred,
        'freq_gt': freq_gt,
    })

In [ ]:
import pandas as pd

df = pd.DataFrame(prediction_results)
df

In [ ]:
# Calculate the mean absolute error
mae = (df['freq_pred'] - df['freq_gt']).abs().mean()
mae

In [ ]:
# Calculate the mean absolute percentage error
mape = ((df['freq_pred'] - df['freq_gt']).abs() / df['freq_gt']).mean() * 100
mape

In [ ]:
import scipy.stats as stats

# Calculate the Pearson correlation coefficient and the p-value for testing non-correlation
corr, p_value = stats.pearsonr(df['freq_pred'], df['freq_gt'])
corr, p_value